<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;

    // 🔹 Базовый класс
    public class SavingsAccount
    {
        // Делегаты и события
        public delegate void AccountEventHandler(string message);
        public event AccountEventHandler OnDeposit;
        public event AccountEventHandler OnWithdraw;
        public event AccountEventHandler OnInterest;

        // Атрибуты
        public int AccountId { get; set; }
        public string OwnerName { get; set; }
        public decimal Balance { get; protected set; }
        public double InterestRate { get; set; }
        public string Currency { get; set; }
        public List<string> Transactions { get; set; } = new List<string>();

        // Конструктор
        public SavingsAccount(int id, string owner, decimal balance, double rate, string currency)
        {
            AccountId = id;
            OwnerName = owner;
            Balance = balance;
            InterestRate = rate;
            Currency = currency;
        }

        // Методы для вызова событий (protected, чтобы наследники могли использовать)
        protected void RaiseDeposit(string msg) => OnDeposit?.Invoke(msg);
        protected void RaiseWithdraw(string msg) => OnWithdraw?.Invoke(msg);
        protected void RaiseInterest(string msg) => OnInterest?.Invoke(msg);

        // Методы
        public virtual void Deposit(decimal amount)
        {
            Balance += amount;
            Transactions.Add($"Внесено {amount} {Currency}");
            RaiseDeposit($"{OwnerName} внес {amount} {Currency}. Баланс: {Balance}");
        }

        public virtual void Withdraw(decimal amount)
        {
            if (amount <= Balance)
            {
                Balance -= amount;
                Transactions.Add($"Снято {amount} {Currency}");
                RaiseWithdraw($"{OwnerName} снял {amount} {Currency}. Баланс: {Balance}");
            }
            else
            {
                Console.WriteLine("Недостаточно средств!");
            }
        }

        public virtual void CalculateInterest()
        {
            decimal interest = Balance * (decimal)(InterestRate / 100);
            Balance += interest;
            Transactions.Add($"Начислены проценты {interest} {Currency}");
            RaiseInterest($"{OwnerName} получил проценты {interest} {Currency}. Баланс: {Balance}");
        }
    }

    // 🔹 Студенческий счёт
    public class StudentAccount : SavingsAccount
    {
        public int YearOfStudy { get; set; }
        public string University { get; set; }
        public string StudentId { get; set; }

        public StudentAccount(int id, string owner, decimal balance, double rate, string currency,
                              int year, string university, string studentId)
            : base(id, owner, balance, rate, currency)
        {
            YearOfStudy = year;
            University = university;
            StudentId = studentId;
        }

        public override void CalculateInterest()
        {
            decimal interest = Balance * (decimal)((InterestRate / 2) / 100); // льготная ставка
            Balance += interest;
            Transactions.Add($"Студенческие проценты {interest} {Currency}");
            RaiseInterest($"Студент {OwnerName} получил льготные проценты {interest} {Currency}. Баланс: {Balance}");
        }

        public void GetScholarship(decimal amount)
        {
            Deposit(amount);
            RaiseDeposit($"Студент {OwnerName} получил стипендию {amount} {Currency}. Баланс: {Balance}");
        }
    }

    // 🔹 Премиум счёт
    public class PremiumAccount : SavingsAccount
    {
        public decimal MinimumBalance { get; set; }

        public PremiumAccount(int id, string owner, decimal balance, double rate, string currency, decimal minBalance)
            : base(id, owner, balance, rate, currency)
        {
            MinimumBalance = minBalance;
        }

        public override void Withdraw(decimal amount)
        {
            if (Balance - amount >= MinimumBalance)
            {
                base.Withdraw(amount);
            }
            else
            {
                Console.WriteLine("Нельзя снимать ниже минимального баланса!");
            }
        }

        public void TransferTo(SavingsAccount other, decimal amount)
        {
            if (Balance - amount >= MinimumBalance)
            {
                Withdraw(amount);
                other.Deposit(amount);
                RaiseWithdraw($"{OwnerName} перевёл {amount} {Currency} на счёт {other.OwnerName}");
            }
        }
    }

    // 🔹 Инвестиционный счёт
    public class InvestmentAccount : SavingsAccount
    {
        public decimal PortfolioValue { get; set; }
        public int RiskLevel { get; set; }

        public InvestmentAccount(int id, string owner, decimal balance, double rate, string currency,
                                 decimal portfolio, int risk)
            : base(id, owner, balance, rate, currency)
        {
            PortfolioValue = portfolio;
            RiskLevel = risk;
        }

        public override void Deposit(decimal amount)
        {
            decimal investPart = amount * 0.2m; // 20% в инвестиции
            PortfolioValue += investPart;
            base.Deposit(amount - investPart);
            RaiseDeposit($"{OwnerName} инвестировал {investPart} {Currency} в портфель. Текущий портфель: {PortfolioValue}");
        }
    }

            StudentAccount student = new StudentAccount(1, "Иван", 1000m, 5, "RUB", 2, "ТюмГУ", "ST123");
            PremiumAccount premium = new PremiumAccount(2, "Анна", 5000m, 7, "RUB", 1000m);
            InvestmentAccount invest = new InvestmentAccount(3, "Олег", 10000m, 6, "USD", 2000m, 3);

            // Подписка на события
            student.OnDeposit += msg => Console.WriteLine("[DEPOSIT] " + msg);
            student.OnInterest += msg => Console.WriteLine("[INTEREST] " + msg);
            premium.OnWithdraw += msg => Console.WriteLine("[WITHDRAW] " + msg);
            invest.OnDeposit += msg => Console.WriteLine("[DEPOSIT] " + msg);

            // Демонстрация
            student.GetScholarship(3000);
            student.CalculateInterest();

            premium.Withdraw(4500);
            premium.TransferTo(student, 500);

            invest.Deposit(2000);
            invest.CalculateInterest();


[DEPOSIT] Иван внес 3000 RUB. Баланс: 4000
[DEPOSIT] Студент Иван получил стипендию 3000 RUB. Баланс: 4000
[INTEREST] Студент Иван получил льготные проценты 100.000 RUB. Баланс: 4100.000
Нельзя снимать ниже минимального баланса!
[WITHDRAW] Анна снял 500 RUB. Баланс: 4500
[DEPOSIT] Иван внес 500 RUB. Баланс: 4600.000
[WITHDRAW] Анна перевёл 500 RUB на счёт Иван
[DEPOSIT] Олег внес 1600.0 USD. Баланс: 11600.0
[DEPOSIT] Олег инвестировал 400.0 USD в портфель. Текущий портфель: 2400.0
